In [1]:
try:
    from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, CPCInverter, Pair
    from fastlane_bot.tools.optimizer import CPCArbOptimizer, F, MargPOptimizer, PairOptimizer
    from fastlane_bot.tools.analyzer import CPCAnalyzer
    from fastlane_bot.testing import *

except:
    from tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, CPCInverter, Pair
    from tools.optimizer import CPCArbOptimizer, F, MargPOptimizer, PairOptimizer
    from tools.analyzer import CPCAnalyzer
    from tools.testing import *


import numpy as np
import matplotlib.pyplot as plt

print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Pair))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCArbOptimizer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(MargPOptimizer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(PairOptimizer))

plt.style.use('seaborn-v0_8-dark')
plt.rcParams['figure.figsize'] = [12,6]

imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require, Timer
SimplePair v2.2 (30/Apr/2024)
ConstantProductCurve v3.5+dev01 (01/May/2024)
CPCArbOptimizer v5.1 (15/Sep/2023)
MargPOptimizer v5.3.1 (01/May/2024)
PairOptimizer v6.0.1 (21/Sep/2023)


# Optimizer Testing Convergence Changes [NBTest 100]

## THOR related Tests

this test originates in the `Optimizer_2312_THOR` notebook

In [2]:
curves_as_dicts = [{'k': 4.3078885616238194e+24,
  'x': 1250505254484.4102,
  'x_act': 0,
  'y_act': 344491.8061533139,
  'alpha': 0.5,
  'pair': 'USDC-eB48/THOR-8044',
  'cid': '74181555988764585035015664420125470098056-1',
  'fee': 2000.0,
  'descr': 'carbon_v1 THOR-8044/USDC-eB48 2000',
  'constr': 'carb',
  'params': {'exchange': 'carbon_v1',
   'tknx_dec': 18,
   'tkny_dec': 6,
   'tknx_addr': '0xa5f2211B9b8170F694421f2046281775E8468044',
   'tkny_addr': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'blocklud': 18758319,
   'y': 344491.8061533139,
   'yint': 344491.8061533139,
   'A': 0,
   'B': 1.659765242784964,
   'pa': 2.754820936639097,
   'pb': 2.754820936639097}},
 {'k': 1106096356.8039548,
  'x': 2619874.8519412754,
  'x_act': 2619874.8519412754,
  'y_act': 422.1943487049999,
  'alpha': 0.5,
  'pair': 'THOR-8044/WETH-6Cc2',
  'cid': '0xbf1875da0431343b56ec6295f706e257dbe85696e5270a5bdad005d37cc2fd9c',
  'fee': 0.003,
  'descr': 'sushiswap_v2 THOR-8044/WETH-6Cc2 0.003',
  'constr': 'uv2',
  'params': {'exchange': 'sushiswap_v2',
   'tknx_dec': 18,
   'tkny_dec': 18,
   'tknx_addr': '0xa5f2211B9b8170F694421f2046281775E8468044',
   'tkny_addr': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'blocklud': 18758340}},
 {'k': 1233376864385.0625,
  'x': 54102579.539405,
  'x_act': 54102579.539405,
  'y_act': 22797.00662861641,
  'alpha': 0.5,
  'pair': 'USDC-eB48/WETH-6Cc2',
  'cid': '0x68bd2250b4b44996e193e9e001f74a5e5a31b31fbd0bb7df34c66eb8da7e6be2',
  'fee': 3000.0,
  'descr': 'uniswap_v2 USDC-eB48/WETH-6Cc2 0.003',
  'constr': 'uv2',
  'params': {'exchange': 'uniswap_v2',
   'tknx_dec': 6,
   'tkny_dec': 18,
   'tknx_addr': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'tkny_addr': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'blocklud': 18758413}}]

In [3]:
CC = CPCContainer.from_dicts(curves_as_dicts)
len(CC), len(curves_as_dicts)

(3, 3)

### CPC min range width functionality

In [4]:
cdata = dict(y=100, yint=100, pa=100, pb=100, pair="WETH-6Cc2/USDC-eB48", tkny="USDC-eB48")
c  = CPC.from_carbon(**cdata)
c2 = CPC.from_carbon(**cdata, minrw=1e-2)
c4 = CPC.from_carbon(**cdata, minrw=1e-4)
c6 = CPC.from_carbon(**cdata, minrw=1e-6)
c

ConstantProductCurve(k=400000399924037.75, x=2000000.4998099068, x_act=0, y_act=100, alpha=0.5, pair='WETH-6Cc2/USDC-eB48', cid='None', fee=None, descr=None, constr='carb', params={'y': 100, 'yint': 100, 'A': 0, 'B': 9.999997500001562, 'pa': 100, 'pb': 100, 'minrw': 1e-06})

In [5]:
assert c2.params.minrw==0.01
assert c4.params.minrw==0.0001
assert c6.params.minrw==0.000001
assert c.params.minrw==0.000001

In [6]:
assert iseq(c2.p**2/100**2, 1.01)
assert iseq(c4.p**2/100**2, 1.0001)
assert iseq(c6.p**2/100**2, 1.000001)
assert iseq(c.p, c6.p)

In [7]:
assert iseq(c2.p-100, 0.49875621120, eps=1e-3)
assert iseq(c4.p-100, 0.00499987500, eps=1e-3)
assert iseq(c6.p-100, 0.00004999875, eps=1e-3)
assert iseq((c2.p-100)/(c4.p-100), 99.75373596136635, eps=1e-4)
assert iseq((c4.p-100)/(c6.p-100), 99.99752507444194, eps=1e-4)

### margpoptimizer absolute convergence

In [8]:
cdata = dict(y=100, yint=100, pa=100, pb=100, pair="WETH-6Cc2/USDC-eB48", tkny="USDC-eB48")
c  = CPC.from_carbon(**cdata)
O = MargPOptimizer(CPCContainer([c,c]))
r = O.optimize("USDC-eB48", params=dict(verbose=True, debug=True), result=O.MO_DEBUG)
assert r["crit"]["crit"] is None
assert r["crit"]["eps"] == O.MOEPS
assert r["crit"]["epsa"] == O.MOEPSA
assert r["crit"]["epsaunit"] == O.MOEPSAUNIT
assert r["crit"]["pstart"] is None

[margp_optimizer] targettkn = USDC-eB48
[margp_optimizer] calculating price estimates
[margp_optimizer] tknq=USDC-eB48, tknbs=('WETH-6Cc2',)
[margp_optimizer] error calculating price estimates: [('no price found for 1 pairs', {'WETH-6Cc2/USDC-eB48'}, array([[None]], dtype=object))]
[margp_optimizer] price estimates =  None
[margp_optimizer] crit=rel (eps=1e-06, unit=1, norm=L2)


In [9]:
assert raises(O.optimize, "USDC-eB48", params=dict(crit="meh"), result=O.MO_DEBUG) ==\
  'crit must be self.MOCRITR or self.MOCRITA'
assert raises(O.optimize, "USDC-eB48", params=dict(crit=O.MOCRITA), result=O.MO_DEBUG) == \
  "pstart must be provided if crit is self.MOCRITA"
assert raises(O.optimize, "USDC-eB48", params=dict(crit=O.MOCRITA, pstart=dict(FOO=1))) ==\
  "pstart does not contain all required tokens ['WETH-6Cc2'; pstart={'FOO': 1}, tokens_t=('WETH-6Cc2',)]"
assert raises(O.optimize, "USDC-eB48", params=dict(crit=O.MOCRITA, pstart={"WETH-6Cc2":2000, "USDC-eB48":1})) ==\
  "epsaunit USD not in pstart {'WETH-6Cc2': 2000, 'USDC-eB48': 1}"

In [10]:
r = O.optimize("USDC-eB48", params=dict(
    crit = O.MOCRITA,
    eps = 1e-10,
    epsa = 100,
    epsaunit = "dollah",
    pstart = {"dollah":1, "WETH-6Cc2": 2000, "USDC-eB48":1},
), result=O.MO_DEBUG)
assert r["crit"]["crit"] == O.MOCRITA
assert r["crit"]["eps"] == 1e-10
assert r["crit"]["epsa"] == 100
assert r["crit"]["epsaunit"] == "dollah"
assert r["crit"]["pstart"] == {"dollah":1, "WETH-6Cc2": 2000, "USDC-eB48":1}

In [11]:
print(np.inf)

inf
